# RadQueue AI - Intelligent Radiology Triage Worklist

Interactive web application for reviewing AI-triaged radiology cases.
Displays a prioritized radiology queue (RED > YELLOW > GREEN > PURPLE) with full EMR drill-down.

**Usage:** Run the cell below, select number of cases (1-92), and click Generate Queue.

In [ ]:
# Environment setup — mount Drive in Colab, then ensure we're in the project root
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')
    # Find the project root by locating this notebook's directory on Drive
    import subprocess
    result = subprocess.run(['find', '/content/drive', '-maxdepth', '5', '-name', 'generated_emrs_27b', '-type', 'd'],
                           capture_output=True, text=True, timeout=30)
    _found = [p.replace('/data/generated_emrs_27b', '') for p in result.stdout.strip().split('\n') if p]
    if _found:
        os.chdir(_found[0])
        print(f'✓ Google Drive mounted — project root: {os.getcwd()}')
    else:
        print('⚠ Drive mounted but project not found. Run from the notebooks/ folder.')
except ImportError:
    # Running locally — ensure we're in the project root (one level above notebooks/)
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
    print(f'Local environment — project root: {os.getcwd()}')

In [ ]:
# ==============================================================================
# RadQueue AI - Radiology Triage Worklist Web App
# ==============================================================================
import pandas as pd
import json
import os
import math
from IPython.display import HTML, display

# --- Data Loading ---
# Data directory — relative to project root (set by the setup cell above)
_data_rel = os.path.join('data', 'generated_emrs_27b')
_candidates = [
    _data_rel,                                                                   # from project root (normal case)
    os.path.join('..', _data_rel),                                               # from notebooks/ subfolder
]
DATA_DIR = next((p for p in _candidates if os.path.isfile(os.path.join(p, 'triage_results.csv'))), None)
if DATA_DIR is None:
    raise FileNotFoundError(
        'Data directory not found (must contain triage_results.csv).\n'
        f'CWD: {os.getcwd()}\n'
        f'Searched: {[os.path.abspath(p) for p in _candidates]}\n'
        'Run the setup cell above first, or set DATA_DIR manually.'
    )

print(f'Loading data from: {DATA_DIR}')

# Load triage results
triage_df = pd.read_csv(os.path.join(DATA_DIR, 'triage_results.csv'))
print(f'Loaded {len(triage_df)} triage results')

# Load all EMR JSON files
emr_data = {}
for i in range(1, 93):
    fp = os.path.join(DATA_DIR, f'RUQ-{i:03d}.json')
    if os.path.exists(fp):
        with open(fp, 'r') as f:
            emr_data[i] = json.load(f)
print(f'Loaded {len(emr_data)} EMR files')

# Build combined dataset, stripping forbidden fields
FORBIDDEN_TRIAGE_COLS = {'actual_diagnosis', 'pattern_type'}
FORBIDDEN_EMR_KEYS = {'source_vignette', 'parsed_vignette', 'generation_metadata', 'source_scenario'}

all_cases = []
for _, row in triage_df.iterrows():
    case_num = int(row['case_num'])
    emr = emr_data.get(case_num, {})

    # Strip forbidden EMR keys
    emr_clean = {k: v for k, v in emr.items() if k not in FORBIDDEN_EMR_KEYS}

    # Truncate long clinical notes
    for enc in emr_clean.get('encounter_history', []):
        for note in enc.get('clinical_notes', []):
            txt = note.get('note_text', '')
            if len(txt) > 4000:
                note['note_text'] = txt[:4000] + '\n\n[Note truncated for display]'

    # Build triage info (strip forbidden columns)
    triage_info = {}
    for col in triage_df.columns:
        if col not in FORBIDDEN_TRIAGE_COLS:
            val = row[col]
            if isinstance(val, float) and math.isnan(val):
                val = None
            triage_info[col] = val

    all_cases.append({'triage': triage_info, 'emr': emr_clean})

# Serialize to JSON
cases_json = json.dumps(all_cases, default=str)
print(f'Built {len(all_cases)} combined cases ({len(cases_json):,} bytes)')

# --- HTML Template ---
# Note: The regex patterns in renderNoteText use literal \* which Python warns about
# in f-strings but they work correctly as JavaScript regex in the output HTML.
import warnings
warnings.filterwarnings('ignore', category=SyntaxWarning)

html_template = f"""
<div id="rq-app">
<style>
#rq-app {{
  --bg-primary: #0a0e17;
  --bg-secondary: #111827;
  --bg-tertiary: #1a2332;
  --bg-surface: #243044;
  --bg-hover: #2a3a50;
  --text-primary: #e2e8f0;
  --text-secondary: #94a3b8;
  --text-muted: #64748b;
  --border-color: #1e3048;
  --pri-red: #ef4444;
  --pri-red-bg: rgba(239,68,68,0.10);
  --pri-red-border: rgba(239,68,68,0.30);
  --pri-yellow: #f59e0b;
  --pri-yellow-bg: rgba(245,158,11,0.10);
  --pri-yellow-border: rgba(245,158,11,0.30);
  --pri-green: #22c55e;
  --pri-green-bg: rgba(34,197,94,0.10);
  --pri-green-border: rgba(34,197,94,0.30);
  --pri-purple: #a855f7;
  --pri-purple-bg: rgba(168,85,247,0.10);
  --pri-purple-border: rgba(168,85,247,0.30);
  --accent-cyan: #06b6d4;
  --accent-blue: #3b82f6;
  --flag-high: #ef4444;
  --flag-low: #3b82f6;
  --flag-normal: #22c55e;
  --radius: 8px;
  --font-sans: -apple-system, BlinkMacSystemFont, 'Segoe UI', system-ui, sans-serif;
  --font-mono: 'SF Mono', 'Consolas', 'Courier New', monospace;
  font-family: var(--font-sans);
  background: var(--bg-primary);
  color: var(--text-primary);
  min-height: 850px;
  max-height: 92vh;
  position: relative;
  overflow: hidden;
  border-radius: 12px;
  box-sizing: border-box;
  line-height: 1.5;
  font-size: 14px;
}}
#rq-app *, #rq-app *::before, #rq-app *::after {{ box-sizing: border-box; margin: 0; padding: 0; }}
.rq-screen {{ position: absolute; inset: 0; display: flex; flex-direction: column; opacity: 0; pointer-events: none; transition: opacity 0.5s ease, transform 0.5s ease; transform: translateY(10px); }}
.rq-screen.active {{ opacity: 1; pointer-events: auto; transform: translateY(0); }}
.rq-selector {{ justify-content: center; align-items: center; background: radial-gradient(ellipse at center, #0f1a2e 0%, #0a0e17 70%); }}
.rq-selector-box {{ text-align: center; max-width: 500px; padding: 40px; }}
.rq-logo {{ font-size: 48px; font-weight: 800; letter-spacing: -2px; margin-bottom: 4px; }}
.rq-logo span {{ background: linear-gradient(135deg, #06b6d4, #3b82f6, #a855f7); -webkit-background-clip: text; -webkit-text-fill-color: transparent; background-clip: text; }}
.rq-subtitle {{ color: var(--text-secondary); font-size: 15px; margin-bottom: 40px; letter-spacing: 0.5px; }}
.rq-input-group {{ display: flex; align-items: center; justify-content: center; gap: 12px; margin-bottom: 24px; }}
.rq-input-group label {{ color: var(--text-secondary); font-size: 14px; }}
#rq-case-count {{ width: 100px; padding: 12px 16px; background: var(--bg-tertiary); border: 1px solid var(--border-color); border-radius: var(--radius); color: var(--text-primary); font-size: 20px; font-weight: 600; text-align: center; outline: none; transition: border-color 0.3s; font-family: var(--font-mono); }}
#rq-case-count:focus {{ border-color: var(--accent-cyan); box-shadow: 0 0 0 3px rgba(6,182,212,0.15); }}
.rq-btn-generate {{ width: 100%; padding: 14px 32px; border: none; border-radius: var(--radius); background: linear-gradient(135deg, #06b6d4, #3b82f6); color: white; font-size: 16px; font-weight: 700; cursor: pointer; letter-spacing: 0.5px; transition: transform 0.2s, box-shadow 0.2s; }}
.rq-btn-generate:hover {{ transform: translateY(-1px); box-shadow: 0 4px 20px rgba(6,182,212,0.3); }}
.rq-btn-generate:active {{ transform: translateY(0); }}
.rq-selector-note {{ color: var(--text-muted); font-size: 13px; margin-top: 16px; }}
.rq-input-error {{ color: var(--pri-red); font-size: 13px; min-height: 20px; margin-bottom: 8px; }}
.rq-loading {{ justify-content: center; align-items: center; background: var(--bg-primary); }}
.rq-loading-box {{ text-align: center; }}
.rq-scan-ring {{ position: relative; width: 120px; height: 120px; margin: 0 auto 32px; }}
.rq-scan-ring .ring {{ position: absolute; inset: 0; border-radius: 50%; border: 2px solid transparent; }}
.rq-scan-ring .ring-1 {{ border-top-color: var(--accent-cyan); border-right-color: var(--accent-cyan); animation: rq-spin 1.5s linear infinite; }}
.rq-scan-ring .ring-2 {{ inset: 10px; border-bottom-color: var(--accent-blue); border-left-color: var(--accent-blue); animation: rq-spin 2s linear infinite reverse; }}
.rq-scan-ring .ring-3 {{ inset: 20px; border-top-color: var(--pri-purple); animation: rq-spin 1s linear infinite; }}
.rq-scan-ring .core {{ position: absolute; inset: 30px; border-radius: 50%; background: radial-gradient(circle, rgba(6,182,212,0.3), transparent 70%); animation: rq-pulse 1.5s ease-in-out infinite; }}
.rq-scan-ring .dot {{ position: absolute; width: 6px; height: 6px; border-radius: 50%; background: var(--accent-cyan); }}
.rq-scan-ring .dot-1 {{ top: 0; left: 50%; transform: translateX(-50%); animation: rq-dotpulse 1.2s ease-in-out infinite; }}
.rq-scan-ring .dot-2 {{ bottom: 0; left: 50%; transform: translateX(-50%); animation: rq-dotpulse 1.2s ease-in-out 0.3s infinite; }}
.rq-scan-ring .dot-3 {{ left: 0; top: 50%; transform: translateY(-50%); animation: rq-dotpulse 1.2s ease-in-out 0.6s infinite; }}
.rq-scan-ring .dot-4 {{ right: 0; top: 50%; transform: translateY(-50%); animation: rq-dotpulse 1.2s ease-in-out 0.9s infinite; }}
#rq-loading-text {{ color: var(--text-secondary); font-size: 15px; margin-bottom: 24px; min-height: 24px; }}
.rq-progress-track {{ width: 280px; height: 3px; background: var(--bg-tertiary); border-radius: 2px; margin: 0 auto; overflow: hidden; }}
.rq-progress-fill {{ height: 100%; background: linear-gradient(90deg, var(--accent-cyan), var(--accent-blue)); border-radius: 2px; width: 0%; transition: width 0.3s; }}
@keyframes rq-spin {{ to {{ transform: rotate(360deg); }} }}
@keyframes rq-pulse {{ 0%,100% {{ opacity: 0.4; transform: scale(0.95); }} 50% {{ opacity: 1; transform: scale(1.05); }} }}
@keyframes rq-dotpulse {{ 0%,100% {{ opacity: 0.3; transform: translate(-50%,-50%) scale(0.8); }} 50% {{ opacity: 1; transform: translate(-50%,-50%) scale(1.4); }} }}
.rq-queue {{ background: var(--bg-primary); }}
.rq-queue-header {{ display: flex; align-items: center; justify-content: space-between; padding: 16px 24px; background: var(--bg-secondary); border-bottom: 1px solid var(--border-color); flex-shrink: 0; }}
.rq-queue-title {{ font-size: 20px; font-weight: 700; }}
.rq-queue-title span {{ background: linear-gradient(135deg, #06b6d4, #3b82f6); -webkit-background-clip: text; -webkit-text-fill-color: transparent; background-clip: text; }}
.rq-btn-back {{ padding: 8px 16px; background: var(--bg-tertiary); border: 1px solid var(--border-color); border-radius: 6px; color: var(--text-secondary); cursor: pointer; font-size: 13px; transition: all 0.2s; font-family: var(--font-sans); }}
.rq-btn-back:hover {{ background: var(--bg-hover); color: var(--text-primary); }}
.rq-stats-bar {{ display: flex; gap: 12px; padding: 12px 24px; background: var(--bg-secondary); border-bottom: 1px solid var(--border-color); flex-shrink: 0; flex-wrap: wrap; align-items: center; }}
.rq-stat-pill {{ display: inline-flex; align-items: center; gap: 6px; padding: 4px 12px; border-radius: 20px; font-size: 13px; font-weight: 600; }}
.rq-stat-flag {{ width: 16px; height: 16px; }}
.rq-stat-pill.red {{ background: var(--pri-red-bg); color: var(--pri-red); border: 1px solid var(--pri-red-border); }}
.rq-stat-pill.yellow {{ background: var(--pri-yellow-bg); color: var(--pri-yellow); border: 1px solid var(--pri-yellow-border); }}
.rq-stat-pill.green {{ background: var(--pri-green-bg); color: var(--pri-green); border: 1px solid var(--pri-green-border); }}
.rq-stat-pill.purple {{ background: var(--pri-purple-bg); color: var(--pri-purple); border: 1px solid var(--pri-purple-border); }}
.rq-stat-total {{ color: var(--text-muted); font-size: 13px; margin-left: auto; }}
.rq-queue-list {{ flex: 1; overflow-y: auto; padding: 8px 16px; }}
.rq-row {{ display: grid; grid-template-columns: 44px 1fr; gap: 0; background: var(--bg-tertiary); border-radius: var(--radius); margin-bottom: 6px; cursor: pointer; transition: all 0.2s; border: 1px solid transparent; overflow: hidden; }}
.rq-row:hover {{ background: var(--bg-hover); border-color: var(--border-color); transform: translateY(-1px); box-shadow: 0 2px 8px rgba(0,0,0,0.2); }}
.rq-row-pri {{ display: flex; align-items: center; justify-content: center; }}
.rq-row-flag {{ width: 22px; height: 22px; }}
.rq-row-pri.red {{ background: var(--pri-red-bg); color: var(--pri-red); }}
.rq-row-pri.yellow {{ background: var(--pri-yellow-bg); color: var(--pri-yellow); }}
.rq-row-pri.green {{ background: var(--pri-green-bg); color: var(--pri-green); }}
.rq-row-pri.purple {{ background: var(--pri-purple-bg); color: var(--pri-purple); }}
.rq-row-body {{ padding: 10px 14px; min-width: 0; }}
.rq-row-top {{ display: flex; align-items: center; gap: 16px; flex-wrap: wrap; }}
.rq-row-name {{ font-weight: 700; font-size: 14px; font-family: var(--font-mono); white-space: nowrap; }}
.rq-row-demo {{ color: var(--text-secondary); font-size: 13px; white-space: nowrap; }}
.rq-row-mrn {{ color: var(--text-muted); font-size: 12px; font-family: var(--font-mono); white-space: nowrap; }}
.rq-row-protocol {{ color: var(--accent-cyan); font-size: 13px; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; max-width: 280px; }}
.rq-row-setting {{ color: var(--text-muted); font-size: 12px; margin-left: auto; white-space: nowrap; }}
.rq-row-bottom {{ display: flex; align-items: center; gap: 8px; margin-top: 4px; }}
.rq-row-indication {{ color: var(--text-secondary); font-size: 13px; overflow: hidden; text-overflow: ellipsis; white-space: nowrap; flex: 1; }}
.rq-row-safety {{ color: var(--pri-yellow); font-size: 12px; white-space: nowrap; font-weight: 600; }}
.rq-detail {{ background: rgba(0,0,0,0.6); justify-content: flex-end; flex-direction: row; }}
.rq-detail-backdrop {{ position: absolute; inset: 0; cursor: pointer; }}
.rq-detail-panel {{ position: relative; width: 78%; max-width: 1000px; height: 100%; background: var(--bg-secondary); border-left: 1px solid var(--border-color); display: flex; flex-direction: column; transform: translateX(100%); transition: transform 0.35s ease; overflow: hidden; }}
.rq-detail.active .rq-detail-panel {{ transform: translateX(0); }}
.rq-detail-header {{ padding: 16px 24px; background: var(--bg-primary); border-bottom: 1px solid var(--border-color); flex-shrink: 0; }}
.rq-detail-header-top {{ display: flex; align-items: center; gap: 12px; margin-bottom: 8px; }}
.rq-btn-close {{ padding: 6px 12px; background: var(--bg-tertiary); border: 1px solid var(--border-color); border-radius: 6px; color: var(--text-secondary); cursor: pointer; font-size: 13px; transition: all 0.2s; font-family: var(--font-sans); }}
.rq-btn-close:hover {{ background: var(--bg-hover); color: var(--text-primary); }}
.rq-detail-name {{ font-size: 22px; font-weight: 700; font-family: var(--font-mono); }}
.rq-detail-badge {{ display: inline-flex; align-items: center; gap: 6px; padding: 3px 10px; border-radius: 4px; font-size: 12px; font-weight: 700; letter-spacing: 0.5px; }}
.rq-detail-flag {{ width: 18px; height: 18px; }}
.rq-detail-badge.red {{ background: var(--pri-red-bg); color: var(--pri-red); border: 1px solid var(--pri-red-border); }}
.rq-detail-badge.yellow {{ background: var(--pri-yellow-bg); color: var(--pri-yellow); border: 1px solid var(--pri-yellow-border); }}
.rq-detail-badge.green {{ background: var(--pri-green-bg); color: var(--pri-green); border: 1px solid var(--pri-green-border); }}
.rq-detail-badge.purple {{ background: var(--pri-purple-bg); color: var(--pri-purple); border: 1px solid var(--pri-purple-border); }}
.rq-detail-demos {{ color: var(--text-secondary); font-size: 13px; display: flex; gap: 16px; flex-wrap: wrap; }}
.rq-tabs {{ display: flex; gap: 0; border-bottom: 1px solid var(--border-color); background: var(--bg-primary); flex-shrink: 0; overflow-x: auto; padding: 0 16px; }}
.rq-tab {{ padding: 10px 18px; color: var(--text-muted); font-size: 13px; font-weight: 500; cursor: pointer; border-bottom: 2px solid transparent; transition: all 0.2s; white-space: nowrap; background: none; border-top: none; border-left: none; border-right: none; font-family: var(--font-sans); }}
.rq-tab:hover {{ color: var(--text-secondary); }}
.rq-tab.active {{ color: var(--accent-cyan); border-bottom-color: var(--accent-cyan); }}
.rq-detail-content {{ flex: 1; overflow-y: auto; padding: 20px 24px; }}
.rq-card {{ background: var(--bg-tertiary); border: 1px solid var(--border-color); border-radius: var(--radius); padding: 16px; margin-bottom: 16px; }}
.rq-card-title {{ font-size: 12px; font-weight: 700; color: var(--text-muted); letter-spacing: 0.8px; text-transform: uppercase; margin-bottom: 12px; }}
.rq-card-row {{ display: flex; gap: 8px; margin-bottom: 6px; font-size: 13px; }}
.rq-card-label {{ color: var(--text-muted); min-width: 130px; flex-shrink: 0; }}
.rq-card-value {{ color: var(--text-primary); }}
.rq-safety-card {{ background: rgba(239,68,68,0.06); border: 1px solid var(--pri-red-border); border-radius: var(--radius); padding: 16px; margin-bottom: 16px; }}
.rq-safety-item {{ display: flex; gap: 8px; align-items: flex-start; margin-bottom: 8px; font-size: 13px; }}
.rq-safety-icon {{ color: var(--pri-red); font-weight: 700; flex-shrink: 0; }}
.rq-two-col {{ display: grid; grid-template-columns: 1fr 1fr; gap: 16px; }}
@media (max-width: 700px) {{ .rq-two-col {{ grid-template-columns: 1fr; }} }}
.rq-proto-ordered {{ padding: 8px 12px; background: var(--bg-surface); border-radius: 6px; font-family: var(--font-mono); font-size: 13px; border-left: 3px solid var(--text-muted); margin-bottom: 8px; }}
.rq-proto-recommended {{ padding: 8px 12px; background: rgba(6,182,212,0.08); border-radius: 6px; font-family: var(--font-mono); font-size: 13px; border-left: 3px solid var(--accent-cyan); color: var(--accent-cyan); }}
.rq-lab-panel {{ margin-bottom: 20px; }}
.rq-lab-panel-header {{ font-weight: 700; font-size: 14px; color: var(--accent-cyan); margin-bottom: 6px; padding-bottom: 4px; border-bottom: 1px solid var(--border-color); }}
.rq-lab-table {{ width: 100%; border-collapse: collapse; font-size: 13px; }}
.rq-lab-table th {{ text-align: left; padding: 6px 10px; color: var(--text-muted); font-weight: 600; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px; border-bottom: 1px solid var(--border-color); }}
.rq-lab-table td {{ padding: 5px 10px; border-bottom: 1px solid rgba(30,48,72,0.5); }}
.rq-lab-table tr.flag-h {{ background: rgba(239,68,68,0.06); }}
.rq-lab-table tr.flag-l {{ background: rgba(59,130,246,0.06); }}
.rq-lab-table .val {{ font-family: var(--font-mono); font-weight: 600; }}
.rq-lab-flag {{ display: inline-block; padding: 1px 6px; border-radius: 3px; font-size: 11px; font-weight: 700; }}
.rq-lab-flag.H {{ background: rgba(239,68,68,0.15); color: var(--flag-high); }}
.rq-lab-flag.L {{ background: rgba(59,130,246,0.15); color: var(--flag-low); }}
.rq-vitals-grid {{ display: grid; grid-template-columns: repeat(auto-fill, minmax(140px, 1fr)); gap: 12px; }}
.rq-vital-card {{ background: var(--bg-surface); border-radius: var(--radius); padding: 12px; text-align: center; }}
.rq-vital-label {{ font-size: 11px; color: var(--text-muted); text-transform: uppercase; letter-spacing: 0.5px; margin-bottom: 4px; }}
.rq-vital-value {{ font-size: 24px; font-weight: 700; font-family: var(--font-mono); }}
.rq-vital-unit {{ font-size: 11px; color: var(--text-muted); }}
.rq-vital-value.critical {{ color: var(--pri-red); }}
.rq-vital-value.elevated {{ color: var(--pri-yellow); }}
.rq-vital-value.normal {{ color: var(--pri-green); }}
.rq-note {{ margin-bottom: 16px; border: 1px solid var(--border-color); border-radius: var(--radius); overflow: hidden; }}
.rq-note-header {{ display: flex; justify-content: space-between; align-items: center; padding: 10px 14px; background: var(--bg-surface); cursor: pointer; }}
.rq-note-header:hover {{ background: var(--bg-hover); }}
.rq-note-type {{ font-weight: 700; font-size: 13px; }}
.rq-note-meta {{ color: var(--text-muted); font-size: 12px; }}
.rq-note-body {{ padding: 14px; font-size: 13px; line-height: 1.7; white-space: pre-wrap; display: none; max-height: 500px; overflow-y: auto; }}
.rq-note.open .rq-note-body {{ display: block; }}
.rq-timeline {{ position: relative; padding-left: 24px; }}
.rq-timeline::before {{ content: ''; position: absolute; left: 8px; top: 4px; bottom: 4px; width: 2px; background: var(--border-color); }}
.rq-tl-item {{ position: relative; margin-bottom: 16px; padding-left: 16px; }}
.rq-tl-item::before {{ content: ''; position: absolute; left: -20px; top: 6px; width: 10px; height: 10px; border-radius: 50%; background: var(--bg-surface); border: 2px solid var(--text-muted); }}
.rq-tl-item.current::before {{ background: var(--accent-cyan); border-color: var(--accent-cyan); box-shadow: 0 0 8px rgba(6,182,212,0.4); }}
.rq-tl-date {{ font-family: var(--font-mono); font-size: 12px; color: var(--text-muted); }}
.rq-tl-title {{ font-weight: 600; font-size: 14px; }}
.rq-tl-sub {{ font-size: 12px; color: var(--text-secondary); }}
.rq-tl-badge {{ display: inline-block; padding: 1px 8px; border-radius: 3px; font-size: 11px; font-weight: 600; background: var(--bg-surface); color: var(--text-secondary); margin-right: 4px; }}
.rq-tl-badge.current {{ background: rgba(6,182,212,0.15); color: var(--accent-cyan); }}
.rq-enc-select {{ padding: 6px 12px; background: var(--bg-tertiary); border: 1px solid var(--border-color); border-radius: 6px; color: var(--text-primary); font-size: 13px; margin-bottom: 16px; font-family: var(--font-sans); }}
.rq-med-table {{ width: 100%; border-collapse: collapse; font-size: 13px; }}
.rq-med-table th {{ text-align: left; padding: 6px 10px; color: var(--text-muted); font-weight: 600; font-size: 11px; text-transform: uppercase; letter-spacing: 0.5px; border-bottom: 1px solid var(--border-color); }}
.rq-med-table td {{ padding: 5px 10px; border-bottom: 1px solid rgba(30,48,72,0.5); }}
.rq-img-order {{ background: rgba(6,182,212,0.06); border: 1px solid rgba(6,182,212,0.2); border-radius: var(--radius); padding: 16px; margin-bottom: 16px; }}
.rq-img-order .label {{ color: var(--accent-cyan); font-weight: 700; font-size: 12px; text-transform: uppercase; letter-spacing: 0.5px; margin-bottom: 8px; }}
.rq-impression {{ background: var(--bg-surface); border-left: 3px solid var(--accent-cyan); padding: 10px 14px; border-radius: 0 6px 6px 0; margin-top: 8px; font-style: italic; }}
.rq-sh-grid {{ display: grid; grid-template-columns: repeat(auto-fill, minmax(200px, 1fr)); gap: 10px; }}
.rq-sh-item {{ padding: 10px; background: var(--bg-surface); border-radius: 6px; }}
.rq-sh-item .label {{ font-size: 11px; color: var(--text-muted); text-transform: uppercase; letter-spacing: 0.5px; }}
.rq-sh-item .value {{ font-size: 14px; font-weight: 500; margin-top: 2px; }}
.rq-pill-list {{ display: flex; flex-wrap: wrap; gap: 6px; }}
.rq-pill {{ display: inline-block; padding: 3px 10px; background: var(--bg-surface); border-radius: 20px; font-size: 12px; color: var(--text-secondary); border: 1px solid var(--border-color); }}
.rq-pill.active {{ border-color: var(--accent-cyan); color: var(--accent-cyan); }}
.rq-pill.nkda {{ border-color: var(--pri-green-border); color: var(--pri-green); }}
#rq-app ::-webkit-scrollbar {{ width: 6px; }}
#rq-app ::-webkit-scrollbar-track {{ background: transparent; }}
#rq-app ::-webkit-scrollbar-thumb {{ background: var(--bg-surface); border-radius: 3px; }}
#rq-app ::-webkit-scrollbar-thumb:hover {{ background: var(--bg-hover); }}
.rq-legend {{ margin-bottom: 32px; padding: 16px; background: rgba(6, 182, 212, 0.05); border-radius: var(--radius); border-left: 4px solid var(--accent-cyan); }}
.rq-legend-title {{ font-size: 13px; font-weight: 700; color: var(--accent-cyan); letter-spacing: 0.5px; text-transform: uppercase; margin-bottom: 12px; }}
.rq-legend-flags {{ display: flex; flex-direction: column; gap: 10px; }}
.rq-flag-item {{ display: flex; gap: 12px; align-items: center; font-size: 13px; color: var(--text-secondary); }}
.rq-flag-icon {{ width: 22px; height: 22px; flex-shrink: 0; }}
.rq-flag-text {{ line-height: 1.5; }}
</style>

<div id="rq-screen-selector" class="rq-screen rq-selector active">
  <div class="rq-selector-box">
    <div class="rq-logo"><span>RadQueue AI</span></div>
    <div class="rq-subtitle">Intelligent Radiology Triage Worklist</div>
    <div class="rq-input-group">
      <label for="rq-case-count">Cases to load</label>
      <input type="number" id="rq-case-count" min="1" max="92" value="20" />
    </div>
    <div id="rq-input-error" class="rq-input-error"></div>
    <button class="rq-btn-generate" onclick="RQ.generate()">Generate Queue</button>
    <div class="rq-selector-note">92 total cases available &bull; Randomly sampled &bull; Sorted by priority</div>
  </div>
</div>

<div id="rq-screen-loading" class="rq-screen rq-loading">
  <div class="rq-loading-box">
    <div class="rq-scan-ring">
      <div class="ring ring-1"></div>
      <div class="ring ring-2"></div>
      <div class="ring ring-3"></div>
      <div class="core"></div>
      <div class="dot dot-1"></div>
      <div class="dot dot-2"></div>
      <div class="dot dot-3"></div>
      <div class="dot dot-4"></div>
    </div>
    <div id="rq-loading-text">Initializing AI triage engine...</div>
    <div class="rq-progress-track"><div id="rq-progress-fill" class="rq-progress-fill"></div></div>
  </div>
</div>

<div id="rq-screen-queue" class="rq-screen rq-queue">
  <div class="rq-queue-header">
    <div class="rq-queue-title"><span>RadQueue AI</span></div>
    <button class="rq-btn-back" onclick="RQ.backToSelector()">New Queue</button>
  </div>
  <div class="rq-stats-bar" id="rq-stats-bar"></div>
  <div class="rq-legend" id="rq-legend" style="margin: 0; border-radius: 0; border-left: none; border-bottom: 1px solid var(--border-color); padding: 10px 24px;">
    <div style="display: flex; align-items: center; gap: 6px; margin-bottom: 8px;">
      <span class="rq-legend-title" style="margin-bottom: 0;">Priority Flags</span>
      <span style="color: var(--text-muted); font-size: 11px;">&mdash; ACR Guideline Adherence</span>
      <button id="rq-legend-toggle" onclick="RQ.toggleLegend()" style="margin-left: auto; background: none; border: none; color: var(--text-muted); cursor: pointer; font-size: 12px; font-family: var(--font-sans);">Hide</button>
    </div>
    <div id="rq-legend-body" class="rq-legend-flags" style="flex-direction: row; flex-wrap: wrap; gap: 16px;">
      <span class="rq-flag-item" style="gap: 6px;">
        <img class="rq-flag-icon" style="width:18px;height:18px;" src="data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCAyNCAyNCIgd2lkdGg9IjI0IiBoZWlnaHQ9IjI0Ij48cGF0aCBkPSJNNCAydjIwIiBzdHJva2U9IiNjYmQ1ZTEiIHN0cm9rZS13aWR0aD0iMS41IiBzdHJva2UtbGluZWNhcD0icm91bmQiLz48cGF0aCBkPSJNNCA0IEM4IDIsIDEyIDYsIDE2IDQgTDE2IDEzIEMxMiAxNSwgOCAxMSwgNCAxM1oiIGZpbGw9IiNlZjQ0NDQiLz48L3N2Zz4=" alt="Red flag" />
        <span class="rq-flag-text" style="font-size:12px;"><strong style="color:var(--pri-red);">High:</strong> Inappropriate / Unsafe</span>
      </span>
      <span class="rq-flag-item" style="gap: 6px;">
        <img class="rq-flag-icon" style="width:18px;height:18px;" src="data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCAyNCAyNCIgd2lkdGg9IjI0IiBoZWlnaHQ9IjI0Ij48cGF0aCBkPSJNNCAydjIwIiBzdHJva2U9IiNjYmQ1ZTEiIHN0cm9rZS13aWR0aD0iMS41IiBzdHJva2UtbGluZWNhcD0icm91bmQiLz48cGF0aCBkPSJNNCA0IEM4IDIsIDEyIDYsIDE2IDQgTDE2IDEzIEMxMiAxNSwgOCAxMSwgNCAxM1oiIGZpbGw9IiNmNTllMGIiLz48L3N2Zz4=" alt="Yellow flag" />
        <span class="rq-flag-text" style="font-size:12px;"><strong style="color:var(--pri-yellow);">Medium:</strong> Suboptimal</span>
      </span>
      <span class="rq-flag-item" style="gap: 6px;">
        <img class="rq-flag-icon" style="width:18px;height:18px;" src="data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCAyNCAyNCIgd2lkdGg9IjI0IiBoZWlnaHQ9IjI0Ij48cGF0aCBkPSJNNCAydjIwIiBzdHJva2U9IiNjYmQ1ZTEiIHN0cm9rZS13aWR0aD0iMS41IiBzdHJva2UtbGluZWNhcD0icm91bmQiLz48cGF0aCBkPSJNNCA0IEM4IDIsIDEyIDYsIDE2IDQgTDE2IDEzIEMxMiAxNSwgOCAxMSwgNCAxM1oiIGZpbGw9IiMyMmM1NWUiLz48L3N2Zz4=" alt="Green flag" />
        <span class="rq-flag-text" style="font-size:12px;"><strong style="color:var(--pri-green);">Low:</strong> Appropriate</span>
      </span>
      <span class="rq-flag-item" style="gap: 6px;">
        <img class="rq-flag-icon" style="width:18px;height:18px;" src="data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCAyNCAyNCIgd2lkdGg9IjI0IiBoZWlnaHQ9IjI0Ij48cGF0aCBkPSJNNCAydjIwIiBzdHJva2U9IiNjYmQ1ZTEiIHN0cm9rZS13aWR0aD0iMS41IiBzdHJva2UtbGluZWNhcD0icm91bmQiLz48cGF0aCBkPSJNNCA0IEM4IDIsIDEyIDYsIDE2IDQgTDE2IDEzIEMxMiAxNSwgOCAxMSwgNCAxM1oiIGZpbGw9IiNhODU1ZjciLz48L3N2Zz4=" alt="Purple flag" />
        <span class="rq-flag-text" style="font-size:12px;"><strong style="color:var(--pri-purple);">Insufficient Info</strong></span>
      </span>
    </div>
  </div>
  <div class="rq-queue-list" id="rq-queue-list"></div>
</div>

<div id="rq-screen-detail" class="rq-screen rq-detail">
  <div class="rq-detail-backdrop" onclick="RQ.closeDetail()"></div>
  <div class="rq-detail-panel">
    <div class="rq-detail-header" id="rq-detail-header"></div>
    <div class="rq-tabs" id="rq-tabs"></div>
    <div class="rq-detail-content" id="rq-detail-content"></div>
  </div>
</div>

<script>
(function() {{
  'use strict';
  const ALL_CASES = {cases_json};
  const PRIORITY_ORDER = {{'RED':0,'YELLOW':1,'GREEN':2,'PURPLE':3}};
  const PRIORITY_LABELS = {{
    'RED':'Urgent Review Required — ACR rates this protocol as inappropriate or unsafe',
    'YELLOW':'Manual Review Recommended — ACR rates this protocol as suboptimal; consider alternatives',
    'GREEN':'Appropriate & Aligned — ACR appropriateness rating supports this protocol choice',
    'PURPLE':'Insufficient Information — Cannot assess ACR appropriateness with available data'
  }};
  const TABS = ['Overview','Labs','Vitals','Notes','Imaging','Medications','History'];
  let state = {{ screen:'selector', cases:[], caseIdx:null, tab:'Overview' }};
  const $ = s => document.querySelector(s);
  const $$ = s => document.querySelectorAll(s);
  const esc = s => {{ const d = document.createElement('div'); d.textContent = s||''; return d.innerHTML; }};
  const trunc = (s,n) => s && s.length > n ? s.slice(0,n)+'...' : (s||'');
  const priClass = p => (p||'').toLowerCase();
  const fmtDate = d => {{ if(!d) return ''; try {{ const dt=new Date(d); return dt.toLocaleDateString('en-US',{{month:'short',day:'numeric',year:'numeric'}}); }} catch(e){{ return d; }} }};
  const fmtTime = d => {{ if(!d) return ''; try {{ const dt=new Date(d); return dt.toLocaleTimeString('en-US',{{hour:'2-digit',minute:'2-digit'}}); }} catch(e){{ return ''; }} }};
  const fmtDateTime = d => fmtDate(d) + ' ' + fmtTime(d);
  const FLAG_SRCS = {{
    'RED':'data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCAyNCAyNCIgd2lkdGg9IjI0IiBoZWlnaHQ9IjI0Ij48cGF0aCBkPSJNNCAydjIwIiBzdHJva2U9IiNjYmQ1ZTEiIHN0cm9rZS13aWR0aD0iMS41IiBzdHJva2UtbGluZWNhcD0icm91bmQiLz48cGF0aCBkPSJNNCA0IEM4IDIsIDEyIDYsIDE2IDQgTDE2IDEzIEMxMiAxNSwgOCAxMSwgNCAxM1oiIGZpbGw9IiNlZjQ0NDQiLz48L3N2Zz4=',
    'YELLOW':'data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCAyNCAyNCIgd2lkdGg9IjI0IiBoZWlnaHQ9IjI0Ij48cGF0aCBkPSJNNCAydjIwIiBzdHJva2U9IiNjYmQ1ZTEiIHN0cm9rZS13aWR0aD0iMS41IiBzdHJva2UtbGluZWNhcD0icm91bmQiLz48cGF0aCBkPSJNNCA0IEM4IDIsIDEyIDYsIDE2IDQgTDE2IDEzIEMxMiAxNSwgOCAxMSwgNCAxM1oiIGZpbGw9IiNmNTllMGIiLz48L3N2Zz4=',
    'GREEN':'data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCAyNCAyNCIgd2lkdGg9IjI0IiBoZWlnaHQ9IjI0Ij48cGF0aCBkPSJNNCAydjIwIiBzdHJva2U9IiNjYmQ1ZTEiIHN0cm9rZS13aWR0aD0iMS41IiBzdHJva2UtbGluZWNhcD0icm91bmQiLz48cGF0aCBkPSJNNCA0IEM4IDIsIDEyIDYsIDE2IDQgTDE2IDEzIEMxMiAxNSwgOCAxMSwgNCAxM1oiIGZpbGw9IiMyMmM1NWUiLz48L3N2Zz4=',
    'PURPLE':'data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCAyNCAyNCIgd2lkdGg9IjI0IiBoZWlnaHQ9IjI0Ij48cGF0aCBkPSJNNCAydjIwIiBzdHJva2U9IiNjYmQ1ZTEiIHN0cm9rZS13aWR0aD0iMS41IiBzdHJva2UtbGluZWNhcD0icm91bmQiLz48cGF0aCBkPSJNNCA0IEM4IDIsIDEyIDYsIDE2IDQgTDE2IDEzIEMxMiAxNSwgOCAxMSwgNCAxM1oiIGZpbGw9IiNhODU1ZjciLz48L3N2Zz4='
  }};
  const _flagSrc = p => FLAG_SRCS[p] || FLAG_SRCS['PURPLE'];

  function showScreen(id) {{
    $$('.rq-screen').forEach(s => s.classList.remove('active'));
    const el = $('#rq-screen-' + id);
    if(el) setTimeout(() => el.classList.add('active'), 20);
    state.screen = id;
  }}

  function generate() {{
    const input = $('#rq-case-count');
    const count = parseInt(input.value);
    const errEl = $('#rq-input-error');
    if(isNaN(count) || count < 1 || count > 92) {{ errEl.textContent = 'Please enter a number between 1 and 92'; return; }}
    errEl.textContent = '';
    const shuffled = [...ALL_CASES].sort(() => Math.random() - 0.5);
    state.cases = shuffled.slice(0, count);
    state.cases.sort((a,b) => {{
      const pa = PRIORITY_ORDER[a.triage.triage_priority] ?? 4;
      const pb = PRIORITY_ORDER[b.triage.triage_priority] ?? 4;
      return pa !== pb ? pa - pb : (a.triage.case_num||0) - (b.triage.case_num||0);
    }});
    showScreen('loading');
    runLoading();
  }}

  function runLoading() {{
    const msgs = ['Initializing AI triage engine...','Analyzing patient records...','Cross-referencing ACR criteria...','Evaluating safety parameters...','Generating priority queue...'];
    const textEl = $('#rq-loading-text');
    const fillEl = $('#rq-progress-fill');
    let i = 0;
    fillEl.style.width = '0%';
    const iv = setInterval(() => {{ i++; if(i < msgs.length) textEl.textContent = msgs[i]; fillEl.style.width = Math.min(100, (i+1)*20) + '%'; }}, 550);
    setTimeout(() => {{ clearInterval(iv); fillEl.style.width = '100%'; renderQueue(); showScreen('queue'); }}, 2800);
  }}

  function renderQueue() {{
    const counts = {{RED:0,YELLOW:0,GREEN:0,PURPLE:0}};
    let safetyCount = 0;
    state.cases.forEach(c => {{ const p = c.triage.triage_priority; if(counts[p] !== undefined) counts[p]++; if(c.triage.has_safety_concern === true || c.triage.has_safety_concern === 'True') safetyCount++; }});
    let statsHtml = '';
    if(counts.RED) statsHtml += '<span class="rq-stat-pill red"><img class="rq-stat-flag" src="'+FLAG_SRCS['RED']+'" alt=""/> '+counts.RED+'</span>';
    if(counts.YELLOW) statsHtml += '<span class="rq-stat-pill yellow"><img class="rq-stat-flag" src="'+FLAG_SRCS['YELLOW']+'" alt=""/> '+counts.YELLOW+'</span>';
    if(counts.GREEN) statsHtml += '<span class="rq-stat-pill green"><img class="rq-stat-flag" src="'+FLAG_SRCS['GREEN']+'" alt=""/> '+counts.GREEN+'</span>';
    if(counts.PURPLE) statsHtml += '<span class="rq-stat-pill purple"><img class="rq-stat-flag" src="'+FLAG_SRCS['PURPLE']+'" alt=""/> '+counts.PURPLE+'</span>';
    if(safetyCount) statsHtml += '<span class="rq-stat-pill red">\u26a0 '+safetyCount+' Safety</span>';
    statsHtml += '<span class="rq-stat-total">'+state.cases.length+' cases in queue</span>';
    $('#rq-stats-bar').innerHTML = statsHtml;

    let rowsHtml = '';
    state.cases.forEach((c, idx) => {{
      const t = c.triage, p = c.emr.patient || {{}};
      const pri = priClass(t.triage_priority);
      const name = ((p.last_name||'') + ', ' + (p.first_name||'')).toUpperCase();
      const ageSex = (p.age||'?') + (p.sex ? p.sex.charAt(0) : '');
      const lastEnc = (c.emr.encounter_history||[]).slice(-1)[0];
      const admTime = lastEnc ? fmtTime(lastEnc.encounter.admission_datetime) : '';
      const hasSafety = t.has_safety_concern === true || t.has_safety_concern === 'True';
      rowsHtml += '<div class="rq-row" onclick="RQ.openDetail('+idx+')">'
        + '<div class="rq-row-pri '+pri+'"><img class="rq-row-flag" src="'+_flagSrc(t.triage_priority)+'" alt="'+esc(t.triage_priority||'')+'"/></div>'
        + '<div class="rq-row-body">'
        + '<div class="rq-row-top">'
        + '<span class="rq-row-name">'+esc(name)+'</span>'
        + '<span class="rq-row-demo">'+esc(ageSex)+'</span>'
        + '<span class="rq-row-mrn">'+esc(p.mrn||'')+'</span>'
        + '<span class="rq-row-protocol">'+esc(trunc(t.protocol_ordered,40))+'</span>'
        + '<span class="rq-row-setting">'+esc(t.setting||'')+'</span>'
        + '</div>'
        + '<div class="rq-row-bottom">'
        + '<span class="rq-row-indication">'+esc(trunc(t.provider_indication,80))+'</span>'
        + (hasSafety ? '<span class="rq-row-safety">\u26a0 Safety Alert</span>' : '')
        + (admTime ? '<span class="rq-row-mrn">'+esc(admTime)+'</span>' : '')
        + '</div></div></div>';
    }});
    $('#rq-queue-list').innerHTML = rowsHtml;
  }}

  function openDetail(idx) {{
    state.caseIdx = idx; state.tab = 'Overview';
    const c = state.cases[idx];
    renderDetailHeader(c); renderTabs(); renderTabContent(c);
    showScreen('detail');
  }}
  function closeDetail() {{ showScreen('queue'); }}

  function renderDetailHeader(c) {{
    const t = c.triage, p = c.emr.patient || {{}};
    const pri = priClass(t.triage_priority);
    const name = ((p.last_name||'') + ', ' + (p.first_name||'')).toUpperCase();
    const conf = t.triage_confidence ? Math.round(t.triage_confidence * 100) : '?';
    $('#rq-detail-header').innerHTML = '<div class="rq-detail-header-top">'
      + '<button class="rq-btn-close" onclick="RQ.closeDetail()">&larr; Back</button>'
      + '<span class="rq-detail-name">'+esc(name)+'</span>'
      + '<span class="rq-detail-badge '+pri+'"><img class="rq-detail-flag" src="'+_flagSrc(t.triage_priority)+'" alt=""/> '+esc(t.triage_priority||'')+' &bull; '+conf+'% confidence</span>'
      + '</div><div class="rq-detail-demos">'
      + '<span>'+esc(String(p.age||''))+(p.sex ? p.sex.charAt(0) : '')+'</span>'
      + '<span>DOB: '+esc(fmtDate(p.date_of_birth))+'</span>'
      + '<span>MRN: '+esc(p.mrn||'')+'</span>'
      + '<span>'+esc(p.sex||'')+'</span>'
      + '<span>'+esc(p.insurance||'')+'</span>'
      + '<span>'+esc(t.setting||'')+'</span>'
      + '</div>';
  }}

  function renderTabs() {{
    let h = '';
    TABS.forEach(tab => {{ h += '<button class="rq-tab '+(tab===state.tab?'active':'')+'" onclick="RQ.switchTab(&quot;'+tab+'&quot;)">'+tab+'</button>'; }});
    $('#rq-tabs').innerHTML = h;
  }}

  function switchTab(tab) {{ state.tab = tab; renderTabs(); renderTabContent(state.cases[state.caseIdx]); }}

  function renderTabContent(c) {{
    const el = $('#rq-detail-content');
    switch(state.tab) {{
      case 'Overview': el.innerHTML = renderOverview(c); break;
      case 'Labs': el.innerHTML = renderLabs(c); break;
      case 'Vitals': el.innerHTML = renderVitals(c); break;
      case 'Notes': el.innerHTML = renderNotes(c); break;
      case 'Imaging': el.innerHTML = renderImaging(c); break;
      case 'Medications': el.innerHTML = renderMedications(c); break;
      case 'History': el.innerHTML = renderHistory(c); break;
    }}
    el.querySelectorAll('.rq-note-header').forEach(h => {{ h.addEventListener('click', () => h.parentElement.classList.toggle('open')); }});
  }}

  function renderOverview(c) {{
    const t = c.triage, p = c.emr.patient || {{}};
    const problems = (c.emr.problem_list||[]).filter(x => x.status === 'Active');
    const allergies = c.emr.allergies || [];
    const pri = priClass(t.triage_priority);
    const priLabel = PRIORITY_LABELS[t.triage_priority] || t.triage_priority;
    const warnings = [];
    ['contrast_warning','renal_warning','pregnancy_warning','coagulation_warning','mri_safety_warning','metformin_warning'].forEach(w => {{
      if(t[w] && t[w] !== '' && t[w] !== 'None' && t[w] !== null) warnings.push({{type: w.replace('_warning','').replace('_',' '), text: t[w]}});
    }});
    let safetyHtml = '';
    if(warnings.length > 0) {{
      safetyHtml = '<div class="rq-safety-card"><div class="rq-card-title" style="color:var(--pri-red)">\u26a0 SAFETY ALERTS</div>';
      warnings.forEach(w => {{ safetyHtml += '<div class="rq-safety-item"><span class="rq-safety-icon">\u26a0</span><span><strong>'+esc(w.type)+':</strong> '+esc(w.text)+'</span></div>'; }});
      safetyHtml += '</div>';
    }}
    let h = safetyHtml + '<div class="rq-two-col"><div>';
    h += '<div class="rq-card"><div class="rq-card-title">Clinical Indication</div><div style="font-size:15px;margin-bottom:12px">'+esc(t.provider_indication||'None provided')+'</div>';
    h += '<div class="rq-card-title" style="margin-top:16px">Protocol Ordered</div><div class="rq-proto-ordered">'+esc(t.protocol_ordered||'None')+'</div>';
    if(t.ai_recommended && t.ai_recommended !== t.protocol_ordered) h += '<div class="rq-card-title" style="margin-top:12px">AI Recommended Protocol</div><div class="rq-proto-recommended">'+esc(t.ai_recommended)+'</div>';
    if(t.acr_rating) h += '<div class="rq-card-row" style="margin-top:12px"><span class="rq-card-label">ACR Rating</span><span class="rq-card-value">'+t.acr_rating+'/9</span></div><div class="rq-card-row"><span class="rq-card-label">Appropriateness</span><span class="rq-card-value">'+esc(t.appropriateness||'')+'</span></div>';
    h += '</div>';
    h += '<div class="rq-card"><div class="rq-card-title">Active Problems</div><div class="rq-pill-list">';
    h += problems.length ? problems.map(p => '<span class="rq-pill active">'+esc(p.condition)+'</span>').join('') : '<span class="rq-pill">None documented</span>';
    h += '</div></div>';
    h += '<div class="rq-card"><div class="rq-card-title">Allergies</div><div class="rq-pill-list">';
    h += allergies.length ? allergies.map(a => '<span class="rq-pill">'+esc(typeof a === 'string' ? a : (a.allergen||a.name||JSON.stringify(a)))+'</span>').join('') : '<span class="rq-pill nkda">NKDA</span>';
    h += '</div></div></div><div>';
    h += '<div class="rq-card"><div class="rq-card-title">Triage Decision</div>';
    h += '<div style="margin-bottom:12px"><span class="rq-detail-badge '+pri+'" style="font-size:14px">'+esc(t.triage_priority||'')+'</span></div>';
    h += '<div style="font-size:13px;color:var(--text-secondary);margin-bottom:12px">'+esc(priLabel)+'</div>';
    if(t.classified_variant && t.classified_variant !== 'N/A') {{
      h += '<div class="rq-card-row"><span class="rq-card-label">ACR Variant</span><span class="rq-card-value">'+esc(t.classified_variant)+'</span></div>';
      h += '<div class="rq-card-row"><span class="rq-card-label">Variant Confidence</span><span class="rq-card-value">'+(t.variant_confidence ? Math.round(t.variant_confidence*100)+'%' : 'N/A')+'</span></div>';
    }}
    if(t.variant_reasoning) h += '<div style="margin-top:12px"><div class="rq-card-title">AI Reasoning</div><div style="font-size:13px;color:var(--text-secondary);line-height:1.6">'+esc(t.variant_reasoning)+'</div></div>';
    h += '</div>';
    if(t.issue_notes) h += '<div class="rq-card"><div class="rq-card-title">Explanation</div><div style="font-size:13px;line-height:1.6">'+esc(t.issue_notes)+'</div></div>';
    h += '<div class="rq-card"><div class="rq-card-title">Patient Info</div>';
    h += '<div class="rq-card-row"><span class="rq-card-label">Phone</span><span class="rq-card-value">'+esc(p.phone||'N/A')+'</span></div>';
    h += '<div class="rq-card-row"><span class="rq-card-label">Emergency Contact</span><span class="rq-card-value">'+esc(p.emergency_contact||'N/A')+'</span></div>';
    h += '<div class="rq-card-row"><span class="rq-card-label">Insurance</span><span class="rq-card-value">'+esc(p.insurance||'N/A')+'</span></div>';
    h += '</div></div></div>';
    return h;
  }}

  function renderLabs(c) {{
    const encounters = c.emr.encounter_history || [];
    if(!encounters.length) return '<div class="rq-card">No lab data available</div>';
    let h = '<select class="rq-enc-select" onchange="RQ._labEnc(this.value)" id="rq-lab-enc-select">';
    encounters.forEach((enc, i) => {{
      const label = fmtDate(enc.encounter.admission_datetime) + ' - ' + (enc.encounter.department||enc.encounter.encounter_type||'');
      h += '<option value="'+i+'" '+(i===encounters.length-1?'selected':'')+'>'+esc(label)+'</option>';
    }});
    h += '</select><div id="rq-lab-panels">'+renderLabPanels(encounters[encounters.length - 1])+'</div>';
    return h;
  }}

  function renderLabPanels(enc) {{
    const labs = enc.lab_results || [];
    if(!labs.length) return '<div class="rq-card">No labs for this encounter</div>';
    let h = '';
    labs.forEach(panel => {{
      h += '<div class="rq-lab-panel"><div class="rq-lab-panel-header">'+esc(panel.panel_name)+' <span style="font-weight:400;font-size:12px;color:var(--text-muted)">('+fmtDateTime(panel.timestamp)+')</span></div>';
      h += '<table class="rq-lab-table"><thead><tr><th>Test</th><th>Value</th><th>Unit</th><th>Range</th><th>Flag</th></tr></thead><tbody>';
      (panel.results||[]).forEach(r => {{
        const flag = r.flag || '';
        const flagCls = flag === 'H' ? 'flag-h' : flag === 'L' ? 'flag-l' : '';
        const range = (r.reference_low != null && r.reference_high != null) ? r.reference_low + ' - ' + r.reference_high : '';
        h += '<tr class="'+flagCls+'"><td>'+esc(r.test_name)+'</td><td class="val">'+(r.value != null ? r.value : '')+'</td><td>'+esc(r.unit||'')+'</td><td>'+range+'</td><td>'+(flag ? '<span class="rq-lab-flag '+flag+'">'+flag+'</span>' : '')+'</td></tr>';
      }});
      h += '</tbody></table></div>';
    }});
    return h;
  }}

  function _labEnc(val) {{
    const c = state.cases[state.caseIdx];
    const enc = (c.emr.encounter_history||[])[parseInt(val)];
    if(enc) $('#rq-lab-panels').innerHTML = renderLabPanels(enc);
  }}

  function renderVitals(c) {{
    const encounters = c.emr.encounter_history || [];
    if(!encounters.length) return '<div class="rq-card">No vitals data available</div>';
    let h = '';
    const lastEnc = encounters[encounters.length - 1];
    const vs = (lastEnc.vital_signs || [])[0];
    if(vs) {{
      h += '<div class="rq-card"><div class="rq-card-title">Current Vitals - '+fmtDateTime(vs.timestamp)+'</div><div class="rq-vitals-grid">';
      const tempCls = vs.temperature_f >= 100.4 ? 'critical' : vs.temperature_f >= 99.5 ? 'elevated' : 'normal';
      h += '<div class="rq-vital-card"><div class="rq-vital-label">Temp</div><div class="rq-vital-value '+tempCls+'">'+(vs.temperature_f||'--')+'</div><div class="rq-vital-unit">&deg;F</div></div>';
      const hrCls = vs.heart_rate > 100 ? 'critical' : vs.heart_rate > 90 ? 'elevated' : 'normal';
      h += '<div class="rq-vital-card"><div class="rq-vital-label">Heart Rate</div><div class="rq-vital-value '+hrCls+'">'+(vs.heart_rate||'--')+'</div><div class="rq-vital-unit">bpm</div></div>';
      const bpCls = vs.blood_pressure_systolic > 160 || vs.blood_pressure_diastolic > 100 ? 'critical' : vs.blood_pressure_systolic > 140 ? 'elevated' : 'normal';
      h += '<div class="rq-vital-card"><div class="rq-vital-label">Blood Pressure</div><div class="rq-vital-value '+bpCls+'">'+(vs.blood_pressure_systolic||'--')+'/'+(vs.blood_pressure_diastolic||'--')+'</div><div class="rq-vital-unit">mmHg</div></div>';
      const rrCls = vs.respiratory_rate > 22 ? 'critical' : vs.respiratory_rate > 20 ? 'elevated' : 'normal';
      h += '<div class="rq-vital-card"><div class="rq-vital-label">Resp Rate</div><div class="rq-vital-value '+rrCls+'">'+(vs.respiratory_rate||'--')+'</div><div class="rq-vital-unit">/min</div></div>';
      const o2Cls = vs.oxygen_saturation < 92 ? 'critical' : vs.oxygen_saturation < 95 ? 'elevated' : 'normal';
      h += '<div class="rq-vital-card"><div class="rq-vital-label">SpO2</div><div class="rq-vital-value '+o2Cls+'">'+(vs.oxygen_saturation||'--')+'</div><div class="rq-vital-unit">%</div></div>';
      const painCls = vs.pain_scale >= 7 ? 'critical' : vs.pain_scale >= 4 ? 'elevated' : 'normal';
      h += '<div class="rq-vital-card"><div class="rq-vital-label">Pain Scale</div><div class="rq-vital-value '+painCls+'">'+(vs.pain_scale != null ? vs.pain_scale : '--')+'</div><div class="rq-vital-unit">/10</div></div>';
      h += '</div></div>';
    }}
    const allVitals = lastEnc.vital_signs || [];
    if(allVitals.length > 1) {{
      h += '<div class="rq-card"><div class="rq-card-title">Serial Vitals</div>';
      h += '<table class="rq-lab-table"><thead><tr><th>Time</th><th>Temp</th><th>HR</th><th>BP</th><th>RR</th><th>SpO2</th><th>Pain</th><th>Source</th></tr></thead><tbody>';
      allVitals.forEach(v => {{ h += '<tr><td>'+fmtTime(v.timestamp)+'</td><td>'+(v.temperature_f||'')+'</td><td>'+(v.heart_rate||'')+'</td><td>'+(v.blood_pressure_systolic||'')+'/'+(v.blood_pressure_diastolic||'')+'</td><td>'+(v.respiratory_rate||'')+'</td><td>'+(v.oxygen_saturation||'')+'</td><td>'+(v.pain_scale != null ? v.pain_scale : '')+'</td><td>'+esc(v.source||'')+'</td></tr>'; }});
      h += '</tbody></table></div>';
    }}
    if(encounters.length > 1) {{
      h += '<div class="rq-card"><div class="rq-card-title">Historical Vitals</div>';
      h += '<table class="rq-lab-table"><thead><tr><th>Date</th><th>Encounter</th><th>Temp</th><th>HR</th><th>BP</th><th>SpO2</th></tr></thead><tbody>';
      encounters.slice(0,-1).reverse().forEach(enc => {{ const v = (enc.vital_signs||[])[0]; if(v) h += '<tr><td>'+fmtDate(enc.encounter.admission_datetime)+'</td><td>'+esc(enc.encounter.department||'')+'</td><td>'+(v.temperature_f||'')+'</td><td>'+(v.heart_rate||'')+'</td><td>'+(v.blood_pressure_systolic||'')+'/'+(v.blood_pressure_diastolic||'')+'</td><td>'+(v.oxygen_saturation||'')+'</td></tr>'; }});
      h += '</tbody></table></div>';
    }}
    return h;
  }}

  function renderNotes(c) {{
    const encounters = c.emr.encounter_history || [];
    if(!encounters.length) return '<div class="rq-card">No notes available</div>';
    let h = '';
    encounters.slice().reverse().forEach((enc, encIdx) => {{
      const isLast = encIdx === 0;
      const notes = enc.clinical_notes || [];
      if(!notes.length) return;
      h += '<div class="rq-card"><div class="rq-card-title">'+fmtDate(enc.encounter.admission_datetime)+' - '+esc(enc.encounter.department||enc.encounter.encounter_type||'')+(isLast ? ' <span style="color:var(--accent-cyan)">(Current)</span>' : '')+'</div>';
      notes.forEach(note => {{
        h += '<div class="rq-note '+(isLast ? 'open' : '')+'">'
          + '<div class="rq-note-header"><span class="rq-note-type">'+esc(note.note_type||'Note')+'</span><span class="rq-note-meta">'+esc(note.author||'')+' &bull; '+fmtTime(note.timestamp)+'</span></div>'
          + '<div class="rq-note-body">'+renderNoteText(note.note_text||'')+'</div></div>';
      }});
      h += '</div>';
    }});
    return h;
  }}

  function renderNoteText(text) {{
    let s = esc(text);
    s = s.replace(/[*][*](.*?)[*][*]/g, '<strong>$1</strong>');
    s = s.replace(/^[*] /gm, '&bull; ');
    s = s.replace(/\\n/g, '<br>');
    return s;
  }}

  function renderImaging(c) {{
    const encounters = c.emr.encounter_history || [];
    let h = '';
    const lastEnc = encounters[encounters.length - 1];
    if(lastEnc) {{
      const orders = lastEnc.imaging_orders || [];
      if(orders.length) {{
        h += '<div class="rq-card-title">Current Imaging Orders</div>';
        orders.forEach(o => {{
          h += '<div class="rq-img-order"><div class="label">Active Order</div>'
            + '<div style="font-size:16px;font-weight:700;margin-bottom:8px">'+esc(o.modality||'')+' '+esc(o.body_region||'')+' '+esc(o.contrast||'')+'</div>'
            + '<div class="rq-card-row"><span class="rq-card-label">Indication</span><span class="rq-card-value">'+esc(o.indication||'')+'</span></div>'
            + '<div class="rq-card-row"><span class="rq-card-label">Urgency</span><span class="rq-card-value">'+esc(o.urgency||'')+'</span></div>'
            + '<div class="rq-card-row"><span class="rq-card-label">Status</span><span class="rq-card-value">'+esc(o.status||'')+'</span></div>'
            + '<div class="rq-card-row"><span class="rq-card-label">Provider</span><span class="rq-card-value">'+esc(o.ordering_provider||'')+'</span></div>'
            + '<div class="rq-card-row"><span class="rq-card-label">Order Time</span><span class="rq-card-value">'+fmtDateTime(o.order_datetime)+'</span></div></div>';
        }});
      }}
    }}
    let hasReports = false;
    encounters.forEach(enc => {{
      (enc.imaging_reports || []).forEach(r => {{
        if(!hasReports) {{ h += '<div class="rq-card-title" style="margin-top:20px">Prior Imaging Reports</div>'; hasReports = true; }}
        h += '<div class="rq-card"><div style="font-weight:700;margin-bottom:8px">'+esc(r.modality||'')+' '+esc(r.body_region||'')+' - '+fmtDate(r.report_datetime||r.timestamp)+'</div>';
        if(r.radiologist) h += '<div class="rq-card-row"><span class="rq-card-label">Radiologist</span><span class="rq-card-value">'+esc(r.radiologist)+'</span></div>';
        if(r.technique) h += '<div style="margin:8px 0;font-size:13px;color:var(--text-secondary)"><strong>Technique:</strong> '+esc(r.technique)+'</div>';
        if(r.findings) h += '<div style="margin:8px 0;font-size:13px"><strong>Findings:</strong> '+esc(r.findings)+'</div>';
        if(r.impression) h += '<div class="rq-impression"><strong>Impression:</strong> '+esc(r.impression)+'</div>';
        h += '</div>';
      }});
    }});
    if(!h) h = '<div class="rq-card">No imaging orders or reports available</div>';
    return h;
  }}

  function renderMedications(c) {{
    const meds = c.emr.current_medications || {{}};
    const home = meds.home_medications || [];
    const inpt = meds.inpatient_medications || [];
    let h = '';
    if(home.length) {{
      h += '<div class="rq-card"><div class="rq-card-title">Home Medications</div>';
      h += '<table class="rq-med-table"><thead><tr><th>Medication</th><th>Dose</th><th>Route</th><th>Frequency</th><th>Indication</th></tr></thead><tbody>';
      home.forEach(m => {{ h += '<tr><td style="font-weight:600">'+esc(m.name||'')+'</td><td>'+esc(m.dose||'')+'</td><td>'+esc(m.route||'')+'</td><td>'+esc(m.frequency||'')+'</td><td style="color:var(--text-secondary)">'+esc(m.indication||'')+'</td></tr>'; }});
      h += '</tbody></table></div>';
    }}
    if(inpt.length) {{
      h += '<div class="rq-card"><div class="rq-card-title" style="color:var(--accent-cyan)">Inpatient Medications</div>';
      h += '<table class="rq-med-table"><thead><tr><th>Medication</th><th>Dose</th><th>Route</th><th>Frequency</th><th>Indication</th></tr></thead><tbody>';
      inpt.forEach(m => {{ h += '<tr><td style="font-weight:600">'+esc(m.name||'')+'</td><td>'+esc(m.dose||'')+'</td><td>'+esc(m.route||'')+'</td><td>'+esc(m.frequency||'')+'</td><td style="color:var(--text-secondary)">'+esc(m.indication||'')+'</td></tr>'; }});
      h += '</tbody></table></div>';
    }}
    if(!home.length && !inpt.length) h = '<div class="rq-card">No medications documented</div>';
    return h;
  }}

  function renderHistory(c) {{
    let h = '';
    const surg = c.emr.surgical_history || [];
    if(surg.length) {{
      h += '<div class="rq-card"><div class="rq-card-title">Surgical History</div>';
      surg.forEach(s => {{ h += '<div class="rq-card-row"><span class="rq-card-label">'+esc(s.procedure||'')+'</span><span class="rq-card-value">'+(s.year||'')+' '+(s.notes ? '- '+esc(s.notes) : '')+'</span></div>'; }});
      h += '</div>';
    }}
    const fam = c.emr.family_history || [];
    if(fam.length) {{
      h += '<div class="rq-card"><div class="rq-card-title">Family History</div><div class="rq-pill-list">';
      fam.forEach(f => {{ h += '<span class="rq-pill">'+esc(typeof f === 'string' ? f : JSON.stringify(f))+'</span>'; }});
      h += '</div></div>';
    }}
    const sh = c.emr.social_history;
    if(sh) {{
      h += '<div class="rq-card"><div class="rq-card-title">Social History</div><div class="rq-sh-grid">';
      [['Smoking',sh.smoking_status],['Alcohol',sh.alcohol_use],['Drug Use',sh.drug_use],['Occupation',sh.occupation],['Living Situation',sh.living_situation]].forEach(([label,val]) => {{
        if(val) h += '<div class="rq-sh-item"><div class="label">'+label+'</div><div class="value">'+esc(val)+'</div></div>';
      }});
      h += '</div></div>';
    }}
    const encounters = c.emr.encounter_history || [];
    if(encounters.length) {{
      h += '<div class="rq-card"><div class="rq-card-title">Encounter Timeline</div><div class="rq-timeline">';
      encounters.slice().reverse().forEach((enc, i) => {{
        const isCurrent = i === 0, e = enc.encounter;
        h += '<div class="rq-tl-item '+(isCurrent ? 'current' : '')+'">'
          + '<div class="rq-tl-date">'+fmtDate(e.admission_datetime)+'</div>'
          + '<div class="rq-tl-title"><span class="rq-tl-badge '+(isCurrent ? 'current' : '')+'">'+esc(e.encounter_type||'')+'</span> '+esc(e.department||'')+'</div>'
          + '<div class="rq-tl-sub">'+esc(e.chief_complaint||'')+' &rarr; '+esc(e.disposition||'')+'</div>'
          + '<div class="rq-tl-sub">Provider: '+esc(e.attending_provider||'')+' &bull; '+esc(e.facility||'')+'</div></div>';
      }});
      h += '</div></div>';
    }}
    if(!h) h = '<div class="rq-card">No history available</div>';
    return h;
  }}

  function backToSelector() {{ showScreen('selector'); }}
  function toggleLegend() {{
    const body = $('#rq-legend-body');
    const btn = $('#rq-legend-toggle');
    if(body.style.display === 'none') {{ body.style.display = 'flex'; btn.textContent = 'Hide'; }}
    else {{ body.style.display = 'none'; btn.textContent = 'Show'; }}
  }}
  window.RQ = {{ generate, openDetail, closeDetail, switchTab, backToSelector, _labEnc, toggleLegend }};
}})();
</script>
</div>
"""

display(HTML(html_template))
print('RadQueue AI loaded successfully.')

In [ ]:
# Save RadQueue as a standalone HTML file — open in any browser
# Run this after cell 2

try:
    full_html = '<!DOCTYPE html>\n<html lang="en">\n<head>\n'
    full_html += '<meta charset="UTF-8">\n'
    full_html += '<meta name="viewport" content="width=device-width, initial-scale=1.0">\n'
    full_html += '<title>RadQueue AI — Radiology Triage Worklist</title>\n'
    full_html += '<style>body { margin: 0; padding: 0; background: #0a0e17; }</style>\n'
    full_html += '</head>\n<body>\n'
    full_html += html_template
    full_html += '\n</body>\n</html>'

    with open('radqueue_app.html', 'w') as f:
        f.write(full_html)

    print(f'\u2713 Saved: radqueue_app.html ({len(full_html):,} bytes)')
    print('  Open this file in your browser to use the app')

    try:
        from google.colab import files
        files.download('radqueue_app.html')
        print('\u2713 Downloading to your computer...')
    except ImportError:
        pass

except NameError:
    print('\u26a0 Run cell 2 first to generate the app')